# Binding expressions

The metalanguage supports a number of operators that can bind variables, with the general syntax:

    Op var: body

For example:

In [ ]:
%te Exists x_e: P_<e,t>(x)

In summary, these operators are:

* `L` (class: `meta.core.LFun`): lambda expressions
* `Exists` (class: `meta.quantifiers.Exists`), `Forall` (class: `meta.quantifiers.Forall`), and `ExistsExact` (class: `meta.quantifiers.ExistsExact`): quantifiers
* `Iota` (class `meta.quantifiers.Iota`): unique reference by condition
* `Set` (class `meta.sets.ConditionSet`): set construction by condition

The behavior is relatively diverse, with the common thread that there is a bound/dependent variable whose identity is shared in the scope of the binder. Type inference will require that distinct instances of these variable can have a unified type (and fail if not). If type inferences needs to adjust the type of any variable the whole expression most follow (and will fail if it can't). The scope of a binding operator on the right continues until explicitly ended, so use parentheses to indicate scope.

Several of these operators, the quantifiers and `Iota`, support an explicit restrictor, i.e. a binary quantifier notation, by providing a set following the variable:

In [ ]:
%te Exists x_e << {_c1, _c2}: P_<e,t>(x)

## Lambda expressions

* Metalanguage operator name: The parser accepts `L`, `λ`, `Lambda`, and `lambda` for this operator.
* class: **`lamb.meta.core.LFun`**

A lambda expression is the main metalanguage tool for describing a function. Lambda expressions have already been introduced, and this document won't itself attempt to explain the notion of a function generally or the lambda calculus, but rather discuss various details of lambda expressions. Given some variable of type `X`, and a body of type `Y`, the type of an `LFun` is `<X,Y>`. Like all binding expressions, the binder names a variable which is then considered bound if it appears in the body of the expression:

In [ ]:
%te L x_n : x + 2

The semantics of such expressions rests primarily in the notions of application and (beta) reduction. The standard parentheses notation is used for application; putting together a function with an argument produces a combined expression (class **`lamb.meta.core.ApplicationExpr`**):

In [ ]:
%te (L x_n : x + 2)(10)

The argument's type and the bound variables type must successfully be able to unify, and the resulting `ApplicationExpr` will reflect any type strengthening. In the following example, an argument of type `n` entails that all instances of `x_X` must be strengthened to be type `n`.

In [ ]:
f = %te L x_X: P_<X,t>(x)
display(f, f(10))

### Reduction and simplification for lambda expressions

Lambda expressions are evaluated by means of (beta) reduction, that is, the substitution of bound variables with an argument. From the above example you can see that the metalanguage separates the application and the reduction step, where the latter must be triggered. When working directly with metalanguage expressions, this can be triggered directly via the function `reduce_all()`, or via simplification with the option `reduce=True`. Currently, this option is not a default simplification strategy, though this may change in the future.

In [ ]:
f = %te L x_X: P_<X,t>(x)
display(f(10).reduce_all())
display(f(10).simplify_all(reduce=True))

Note that when working with composition systems in the `lamb.lang` module, reduction *is* generally triggered by default. In addition, the `simplify` parameter to `%te` magics triggers reduction. (You can provide the parameter `reduce` to trigger reduction without other simplification strategies.)

In [ ]:
f(10).reduce_all().derivation

**Alpha conversion.**
In order to avoid collisions with locally free terms, reduction may result in the automatic renaming of bound variables via a process standardly known as *alpha conversion*. In the following example, a variable `y` is applied for the argument named `x`. Just blindly substituting `y` for `x` would result in a collision with the bound `y`, and therefore the bound `y` is automatically renamed to `y1`:

In [ ]:
%te simplify (L x_e : L y_e : P_<(e,e),t>(x,y))(y_e)

It is possible to trigger alpha conversion manually via the function `alpha_convert`:

In [ ]:
te("L x_e : x").alpha_convert('y')

**Multiple reduction**. The standard triggers for reduction amount to calling the function `reduce_all()`, which recursively triggers all possible reductions in an expression (including ones that result from reduction). For example, the following expression requires three successive reduction steps to fully simplify; one for each argument and one resulting from the first two:

In [ ]:
x = %te (L f_X : L g_Y : f(g))(L p_t : ~p)(Q_t & R_t)
x

In [ ]:
# adding the .trace() here shows the recursive substeps
x.reduce_all().derivation.trace()

**Execution notes**. Compiling an `LFun` generates a python `callable` that can then be used like any other function. The bare compiled function expects a compiled argument and does not do type-checking on its argument, though the wrapper functions provide limited dynamic type-checking as a convenience. As a toy example, the following `meta.exec` call is essentially equivalent to writing python `lambda x : x + 10`, with the addition that static type-checking is carried out while parsing the metalanguage expression.

In [ ]:
f = meta.meta.exec(te("L x_n : x + 10"))
f(2)

## Quantifiers and reference

The quantifiers have uniform type behavior: they take a body of type `t`, and a variable of arbitrary type, producing an expression of type `t`. The universal and existential quantifiers are in many ways quite standard implementations. The `ExistsExact` operator is used for indicating unique existence. These operators have a uniform simplification/execution behavior and api. All of them support restrictor notation. All of these support evaluation/simplification and execution, with the important caveat that a guaranteed finite domain is required (either via the type domain or a restrictor).

### Universal quantification / `Forall`

* Metalanguage operator name: `Forall`
* Class: **`lamb.meta.quantifiers.Forall`**

A universally quantified expression is true iff every element in the *domain*, when instantiated in place of the variable, makes the body expression true. If the domain is empty, the expression is said to be *trivially true*. The domain is determined by the variable's type in combination with any provided restrictor.

For example, the following expression is true iff every element of type `e` is not equivalent to the `MetaTerm` element `_c1`.

In [ ]:
%te Forall x_e : ~(x <=> _c1)

The following expression is similar but provides an explicit restrictor, and so therefore has a guaranteed finite domain.

In [ ]:
%te Forall x_e << {_c2, _c3} : ~(x <=> _c1)

**Simplification and execution**. If a universally quantified expression contains no free terms and has a guaranteed finite domain (or involves vacuous binding), it can be fully simplified. The above example is a case in point; simplification will iterate over the domain and evaluate the body expression at each domain element, finding that the expression is true for both `_c2` and `_c3`.

In [ ]:
%te simplify Forall x_e << {_c2, _c3} : ~(x <=> _c1)
_llast.derivation.trace()

The execution conditions are similar, except that where simplification will not do anything, execution will error. One important note is that the type domain is fixed at compile time, so compiled quantifiers are not dynamically sensitive to type domain restrictions.

In [ ]:
meta.meta.exec(te("Forall x_e << {_c2, _c3} : ~(x <=> _c1)"))

In [ ]:
with lamb.errors():
    meta.meta.exec(te("Forall x_e : ~(x <=> _c1)"))

Like either quantifiers, universal quantification with a finite domain supports **elimination**. For the universal case, that means conversion to a sequence of conjunctions. Where full evaluation requires a concrete domain (consisting only of meta-elements), elimination allows for arbitrary domain elements. Because elimination tends to generate longer expressions than it starts with, it isn't triggered by default, but can be triggered by the simplify option `eliminate_quantifiers=True`. The function `eliminate()` on a `Forall` object also directly triggers one elimination step (though not any related further simplification, or recursive elimination). Here's a more complicated example illustrating elimination; in this case one of the domain elements is bound by a lambda, and the other is a constant term.

In [ ]:
te("L z_e : Forall x_e << {A_e, z_e} : ~(x <=> _c1)").simplify_all(eliminate_quantifiers=True)

Exception: when the domain can be inferred to be exactly size 1, elimination is automatically triggered. Domain size 0 cases are also automatically simplified in a way that is equivalent to elimination.

In [ ]:
%te simplify L z_e : Forall x_e << {z_e} : ~(x <=> _c1)

In [ ]:
%te simplify Forall x_e << {} : ~(x <=> _c1)

While the above examples involve an explicit domain restrictor, the behavior with finite type domains is identical. For example, here is a case of elimination with the type domain for `t`:

In [ ]:
te("Forall p_t : Q_<t,t>(p)").eliminate()

### Existential quantification / `Exists`

* Metalanguage operator name: `Exists`
* Class: **`lamb.meta.quantifiers.Exists`**

An existentially quantified expression is true iff there is at least one element in the *domain* that, when instantiated in place of the variable, makes the body expression true. If the domain is empty, the expression is said to be *trivially false*. The domain is determined by the variable's type in combination with any provided restrictor.

For example, the following expression is true iff there is an element of type `e` is not equivalent to the `MetaTerm` element `_c1`. Following that is an example with an explicit domain restrictor.

In [ ]:
%te Exists x_e : ~(x <=> _c1)

In [ ]:
%te Exists x_e << {_c1, _c2, _c3} : ~(x <=> _c1)

**Simplification and execution**. The situation with `Exists` is very similar to `Forall`: if the expression involves a finite, concrete domain and no free terms it can be fully simplified and executed. The verifier that was used to establish truth can generally be seen in the full derivation.

In [ ]:
%te simplify Exists x_e << {_c1, _c2, _c3} : ~(x <=> _c1)
_llast.derivation.trace()

In [ ]:
meta.meta.exec(te("Exists x_e << {_c1, _c2, _c3} : ~(x <=> _c1)"))

This operator also supports **elimination**, in this case, conversion to a sequence of disjunctions. Here's a slightly more involved example than the universal one; in this case elimination leads to subsequent simplification away of one of the disjuncts.

In [ ]:
te("L z_e : Exists x_e << {A_e, z_e, _c1} : ~(_c1 <=> x)").simplify_all(eliminate_quantifiers=True).derivation.trace()

### ExistsExact

* Metalanguage operator name: `ExistsExact`
* Class: **`lamb.meta.quantifiers.ExistsExact`**

The `ExistsExact` operator is used for truth on unique existence. For example, the following expression is true iff there is exactly one entity that is identical to `_c1`.

In [ ]:
%te ExistsExact x_e : (x <=> _c1)

**Simplification and execution**. As usual, this quantifier can't be avaluated against a potentially non-finite domain, but with explicit domain restriction, it can be simplified:

In [ ]:
%te simplify ExistsExact x_e << {_c1, _c2, _c3, _c4} : (x <=> _c1)
_llast.derivation.trace()

This operator implements `eliminate`, but in a slightly different way than the previous cases: since it can be defined in terms of `Exists` and `Forall`, elimination converts to those operators. Further successive elimination may or may not simplify further. This elimination rule will apply unconditionally if triggered.

In [ ]:
te("ExistsExact x_e : (x <=> _c1)").simplify_all(eliminate_quantifiers=True)

In [ ]:
te("ExistsExact x_e << {_c1, _c2} : (x <=> _c1)").simplify_all(eliminate_quantifiers=True).derivation

### Iota

Iota consists of two operators with slightly different semantics:

* Metalanguage operator names: `Iota`, `IotaPartial`
* Classes: **`lamb.meta.quantifiers.Iota`**, **`lamb.meta.quantifiers.IotaPartial`**

In general, `Iota`, given some condition, evaluates to the unique element satisfying that condition. It is a referential counterpart to `ExistsExact`. For example, the following can be read as the unique `x` in the set provided that is identical to `_c1`.

In [ ]:
%te Iota x_e << {_c1, _c2, _c3} : x <=> _c1

**Simplification and execution**. There are two obvious differences from quantifiers: the whole expression is not necessarily type `t`, and the expression won't necessarily be evaluable at all. The two classes implement different strategies for the latter issue. for both classes, an `eliminate()` call has an identical effect to a `simplify()` call.

First, regular `Iota`. The case where regular `Iota` has a unique referent is relatively straightforward:

In [ ]:
%te simplify Iota x_e << {_c1, _c2, _c3} : x <=> _c1
_llast.derivation.trace()

When reference fails for `Iota` during simplification, something unusual for metalanguage elements will happen. The simplification code will raise a `lamb.meta.meta.DomainError` exception:

In [ ]:
%te simplify Iota x_e << {_c2, _c3} : x <=> _c1

In [ ]:
%te simplify Iota x_n << {1, 2, 3} : x < 3

In many cases it may be desireable to produce a valid metalanguage expression even in the face of uniqueness failure. The standard technique in the metalanguage for doing this is to guard it with a `Partial` condition that checks uniqueness. This is exactly what `IotaPartial` does: it is basically a shorthand for a regular iota conditioned on there being a unique element that meets the body condition.

For the success case, the behavior of `IotaPartial` when simplifying is more complicated, but comes to the same result:

In [ ]:
%te simplify IotaPartial x_e << {_c1, _c2, _c3} : x <=> _c1
_llast.derivation.trace()

For the case where uniqueness or existence fails, the result is a `Partial` object with a failed condition. (See the documentation on partiality for more details.)

In [ ]:
%te simplify IotaPartial x_n << {1, 2, 3} : x < 3

When examining the derivation, you can see that the condition is evaluated first as `False`, which prevents the body from being evaluated and raising an error.

In [ ]:
%te simplify IotaPartial x_n << {1, 2, 3} : x < 3
_llast.derivation.trace()

Execution for `Iota` behaves much like simplification, with the usual caveats about execution (no free terms, finiteness, etc). It will return a non-wrapped domain element directly:

In [ ]:
meta.meta.exec(te("Iota x_e << {_c1, _c2, _c3} : x <=> _c1"))

Like simplification, a `DomainError` will be raised on uniqueness failures. This is checked dynamically, not at compile time, with the caveat that (like other quantifiers) any type domain restrictions are fixed at compile time.

In [ ]:
with lamb.errors():
    meta.meta.exec(te("Iota x_n << {1, 2, 3} : x < 3"))

Warning: an executed `IotaPartial` will have essentially the same behavior! This is because a failed condition on a compiled `Partial` object also raises a `DomainError`. Specifically, a compiled `IotaPartial` is implemented via the compiled version of its eliminated version, which is a compiled `Partial`; executing this will dynamically check the truth of a condition, which (in contrast to `Partial` simplification) will raise a `DomainError` on falsity.

In [ ]:
te("IotaPartial x_e << {_c1, _c2, _c3} : x <=> _c1").eliminate()

In [ ]:
with lamb.errors():
    # this is identical to executing the above expression.
    meta.meta.exec(te("IotaPartial x_e << {_c2, _c3} : x <=> _c1"))

## Set expressions

* Metalanguage operator name: `Set`
* Class: **`lamb.sets.ConditionSet`**

Set expressions allow describing a set by condition on a variable. This class does not support domain restriction.

In [ ]:
%te Set x_n : x < 3

`ConditionSet` is primarily documented in the section on sets; see that for more details.

## General API notes on binding operators

This section provides some information on implementation detail that may be relevant to anyone dealing with binding expressions programmatically.

**Binding operator objects and subclassing**.
All binding expressions have an instance of the variable as their first element, and their body as their second element.

In [ ]:
list(te("Forall x_e : P_<e,t>(x)"))

These can also be accessed via the properties `.var_instance` and `.body` respectively; the variable's name and type can be directly accessed via `.varname`/`.vartype`.

Binding operators all derive from an abstract base class **`lamb.meta.core.BindingOp`**. The abstract base subclass of this **`lamb.meta.core.RestrictedBindingOp`** adds in support for restrictors. These two classes provide a substantial amount of general code for implementing binding operators. Implementing a subclass with just syntax involves providing a constructor and some class-level attributes; implementing a subclass with evaluation additionally involves providing `simplify()`, `_compile()`, and optionally `eliminate()`.

All of the built-in binding expressions need exactly one variable. However, the abstract classes `BindingOp` and `RestrictedBindingOp` both support n-ary variables via the constructor. See the Compositional DRT fragment for an example of a `BindingOp` subclass that makes use of this, and more information about how to add new binding operators generally.

This document won't exhaustively go through the api, but here are a few umore seful functions:

* `.bound_variables()`: return the variables that are locally bound in this expression as a set.
* `.vacuous()`: return `True` iff the bound variable(s) do not appear in the body.
* `.finite_safe()`: is it safe to assume that the domain is finite? returns `None` if this can't be determined.
* `.domain_iter()`: return an iterator over domain elements, if this can be determined (for restricted operators, this depends on the form of the restrictor)
* `.domain_cardinality()`: return the cardinality of the domain, if this can be determined. May be `math.inf`.
* `.alpha_convert()`: this was described for lambda expressions above, but works generally on all binding expressions.
* `.join()`: restrictor-less expressions of type `t` and body type `t` support a non-trivial join. Joining a sequence of variables followed by a condition, gets a recursive sequence of the relevant operator terminating in the provided condition as the deepest body. Binding expressions of other types, as well as restricted expressions, only support trivial joins.

In [ ]:
meta.quantifiers.Forall.join([te("x_e"), te("y_e"), te("P_<(e,e),t>(x,y)")])

**Restrictor-specific notes.**

* For operators that support restrictors, their AST argument length is variable. The restrictor is implemented as the *third* argument when present (mismatching the parser order). The property `restrictor` can be used to safely access this for subclasses of `RestrictedBindingOp`; it will return `None` if there is no restrictor.

In [ ]:
len(te("Forall x_e : P_<e,t>(x)"))

In [ ]:
len(te("Forall x_e << {_c1} : P_<e,t>(x)"))

In [ ]:
te("Forall x_e << {_c1} : P_<e,t>(x)").restrictor

* Both the constructor and `copy_local` for relevant classes support this optionality (though note that `copy_local` does not accept the restrictor as a named parameter). This means that you can use `copy_local` to add a restrictor for a class that supports it:

In [ ]:
x = te("Forall x_e : P_<e,t>(x)")
display(x)
x.copy_local(*x, te("{_c1}"))